In [5]:
import ase
import ase.io
from ase.io import xsf
import pandas as pd

In [2]:
from pathlib import Path

pathlist = Path("data/structures/structures_3D_xsf_small/").glob('**/*.asm')

In [3]:
import os

path = 'data/structures/structures_3D_xsf_small/'

folder = os.fsencode(path)

filenames = []
structures = []
uiids = []


for file in os.listdir(folder):
    
    filename = os.fsdecode(file)
    uiid = filename.split('/')[-1].split('.xsf')[0]
    
    with open(path + os.fsdecode(file)) as f:
        structure = ase.io.xsf.read_xsf(f, index=- 1, read_data=False)
        
        structures.append(structure)
        uiids.append(uiid)
        



In [6]:
df = pd.DataFrame(list(zip(uiids, structures)), 
                  columns=['uiid', 'structure'])
df

,uiid,structure
0,ff8adefe-1c6c-4194-ad5f-a73a21360010,"(Atom('H', [0.5068652416, 4.2920579963, 4.7003..."
1,ffee7707-5c8f-4bdb-ac21-632e11f5833d,"(Atom('Si', [3.692364216, 7.327588456, 3.10475..."
2,ff1102df-2cef-4d8d-a601-1c62ce389cce,"(Atom('H', [17.7396590528, 4.2667548, 5.071097..."
3,ff9dc616-0d19-45fd-8e4a-66c96d842780,"(Atom('H', [6.1991436051, 1.8191698, 1.9452027..."
4,ffadf13b-a453-499f-9f68-ea9be02f00b9,"(Atom('H', [11.5511454, 5.8289976, 18.04741828..."
...,...,...
414,ff1f7fa0-68b8-4c8c-aaf2-857e9b3fc5bc,"(Atom('H', [1.20074746, 0.00472392, 17.7299712..."
415,ffb7329b-5ce6-4d87-85e0-269eb8fdb6f8,"(Atom('H', [6.6610976, 2.62499958, 2.582372], ..."
416,ff6571e8-141e-4230-a505-b97d9944bd21,"(Atom('U', [0.7975425475, 0.0, 2.5187476819], ..."
417,ffa2c42c-eb69-429e-be73-2c32e38b41d6,"(Atom('Cs', [-0.81297208, 15.897426, 1.5659158..."
